In [ ]:
import PIL 
import numpy as np
from collections import defaultdict as dic
from PIL import Image as im


# =================================================================
# =================== Criacao de funcoes ==========================
# =================================================================

# Parte simples achar por cor as informações pedidas
# Funcao que retorna duas informacoes, 
# numero de estrelas e numero de meteoro
# Util para validacao de resultados por isso
# essa funcao foi feita.
def star_meteor(fig_, b_sel_=0):
    # Se o b_sel_ = 0 nao sai a informacao de agua
    # se o b_sel_ = 1 sai essa informacao extra
    # Cria o dicionario que vai ser usado para contar
    by_li = dic(int)
    for pix_ in fig_.getdata():
         by_li[pix_] += 1
    # Sai com as informacoes do dicionario usando a entrada
    # igual ao que se espera achar
    if b_sel_ == 0:
        return by_li[(255,255,255,255)], by_li[(255,0,0,255)]
    else:
        return by_li[(255,255,255,255)], by_li[(255,0,0,255)], by_li[(0,0,255,255)]

    
# Esta funcao serve para sempre que quer ver, salva e 
# abrir uma imagem
def abridor(poderia_ser_, de_garrafa_):
    # poderia_ser_ = array da figura
    # de_garrafa_ = nome na qual a figura vai ser gravada (string)
    fig_imp_ = im.fromarray(np.uint8(poderia_ser_))
    fig_imp_.save(de_garrafa_,'PNG',quality=100)
    fig_imp_.show()


# Função para reduzir array, tirando linhas sem importancia 
# como as apenas com ceu e deixando as outras
def reducerr(arr_red_):
    # Parte que retira o ceu sem estrela ou meteoro
    coun_ = []
    for i_ in range(len(arr_red_)):
        # Essa variavel serve para comparar se tudo vai estar igual
        temp_ = arr_red_[i_][0]
        # Essa variavel serve para o caso nao tenha variacao, a 
        # posicao de i_ seja guardada
        situ_ = 0
        
        # Fazendo o teste da forma menos eficiente porem mais simples
        for j_ in range(len(arr_red_[0])):
            if arr_red_[i_][j_,0] != temp_[0]:
                break
            if arr_red_[i_][j_,1] != temp_[1]:
                break
            if arr_red_[i_][j_,2] != temp_[2]:
                break
            if arr_red_[i_][j_,3] != temp_[3]:
                break
            if j_ == (len(arr_red_[0])-1):
                situ_ = 1
        # Guarda em coun_ a posicao que nao teve variacao de cor
        if situ_ == 1:
            coun_.append(i_)
    # Essa parte apaga as informacoes que estao nas linhas listadas
    # em coun_
    for i_ in reversed(range(len(coun_))):
        deleter_ = coun_[i_]
        arr_red_ = np.delete(arr_red_,(deleter_), axis=0)
    
    # Image open/converter/save
    abridor(arr_red_,'imagem_p1.png')

    # Parte que retira o solo pos primeira linha azul que seja igual
    # segue ao mesmo raciocinio anterior so 
    find_blue_ = []
    # Define em blue_ a cor de azul RGBA = #0000FFFF
    blue_ = (0,0,255,255)
    for i_ in range(len(arr_red_)):
        for j_ in range(len(arr_red_[0])):
            if np.all(blue_ == arr_red_[i_][j_]):
                # Guarda informacao de de azul foi achado (posicao)
                find_blue_.append(i_)
                break
    
    # Para manter uma linha que encontre azul a ultima a ser apagada foi eliminada
    find_blue_ = np.delete(find_blue_,(0), axis=0) # retirando a linha 1
    
    # Apaga as linhas que tem azul e nao estao sendo usadas
    for i_ in reversed(range(len(find_blue_))):
        deleter_ = find_blue_[i_]
        arr_red_ = np.delete(arr_red_,(deleter_), axis=0)

    # Image open/converter/save
    abridor(arr_red_,'imagem_p2.png')

    # Apaga as linhas abaixo da ultima linha que existe agua
    tot_ = len(arr_red_) - find_blue_[0]
    for i_ in reversed(range(1,tot_)):
        deleter_ = find_blue_[0] + i_
        arr_red_ = np.delete(arr_red_,(deleter_), axis=0)
    
    # Image open/converter/save
    abridor(arr_red_,'imagem_p3.png')
    
    # Esta parte serve para remover as partes que nao possuem 
    # estrela, apenas terra e ceu
    last_white_or_red_ = 0
    # Definindo as variaveis referente as cores
    white_ = (255,255,255,255)
    red_ = (255,0,0,255)
    for i_ in range(len(arr_red_)):
        for j_ in range(len(arr_red_[0])):
            if np.all(white_ == arr_red_[i_][j_]) or np.all(red_ == arr_red_[i_][j_]):
                # Quando a linha tiver ou estrela ou meteoro 
                # seu index é guardado em last_white_o_red_
                # e para o loop para eviar loops extras
                last_white_or_red_ = i_
                break
    # Comeca o processo de exclusao a variavel coun 
    # recebe a informacao para comecar a esclusao
    coun = len(arr_red_) - last_white_or_red_ - 2
    for i_ in reversed(range(1,coun)):
        deleter_ = last_white_or_red_ + i_
        arr_red_ = np.delete(arr_red_,(deleter_), axis=0)

    # Image open/converter/save
    abridor(arr_red_,'imagem_p4.png')
    return arr_red_


# Esta funcao testa se tem a cor na posicao e volta um int para cada cor
def color_test(pos_test_,r=0,b=0,w=0):
    # pos_test_ precisa ser no formato RGBA
    # se quiser vermelho r = 1
    #               azul b = 1
    #             branco w = 1

    color_test_exit_ = 0
    if r == 1:
        red_ = np.uint8((255,0,0,255))
        if np.all(red_ == pos_test_):
            color_test_exit_ = 1
            #print(pos_test_)
    if b == 1:
        blue_ = np.uint8((0,0,255,255))
        if np.all(blue_ == pos_test_):
            color_test_exit_ = 2
            #print(pos_test_)
    if w == 1:
        white_ = np.uint8((255,255,255,255))
        if np.all(white_ == pos_test_):
            color_test_exit_ = 3
            #print(pos_test_)
    return color_test_exit_ 


# Cria uma imagem totalmente preta para que possa ser usada
# funcao lenta mas que serviu no momento que eu precisei
def black_image_arr(ima_biar_):
    black_biar_ = ima_biar_.copy()
    black_biar_ = np.asarray(black_biar_)
    black_c_biar_ = np.uint8((0,0,0,255))
    if len(black_biar_) != 1:
        for i_biar_ in range(len(black_biar_)):
            for j_biar_ in range(len(black_biar_[0])):
                black_biar_[i_biar_][j_biar_] = black_c_biar_
    return black_biar_


# Esta funcao transforma o mapa todo em vermalho, branco e preto
# Exceto a linha de agua
def white_red_black(mar_wrb_):
    exit_arr_wrb_ = black_image_arr(mar_wrb_)
    for i_wrb_ in range(len(mar_wrb_)):
        for j_wrb_ in range(len(mar_wrb_[0])):
            white_ = (255,255,255,255)
            blue_ = (0,0,255,255)
            red_ = (255,0,0,255)
            if np.all(red_ == mar_wrb_[i_wrb_][j_wrb_]):
                exit_arr_wrb_[i_wrb_][j_wrb_] = mar_wrb_[i_wrb_][j_wrb_]
            if np.all(blue_ == mar_wrb_[i_wrb_][j_wrb_]):
                exit_arr_wrb_[i_wrb_][j_wrb_] = mar_wrb_[i_wrb_][j_wrb_]
            if np.all(white_ == mar_wrb_[i_wrb_][j_wrb_]):
                exit_arr_wrb_[i_wrb_][j_wrb_] = mar_wrb_[i_wrb_][j_wrb_]
    return exit_arr_wrb_


# "TRUNCAR" IMAGEM
# Essa parte foi feita para compactar as estrelas e meteoros nas 
# primeiras linhas, nesse caso nas duas primeiras
def short_image(mar_si_):
    new_si_ = black_image_arr(mar_si_)
    new_si_ = np.delete(new_si_,(len(new_si_)-1), axis=0) # apaga ultima linha que esta preta
    height_arr_ = len(mar_si_)-2 # linha com azul
    new_si_[height_arr_] = mar_si_[height_arr_]
    height_arr_ -= 1
    width_arr_ = len(mar_si_[0])-1
    fig_temp_ver_ = im.fromarray(black_image_arr(mar_si_)).resize((1,height_arr_))
    fig_temp_hor_ = im.fromarray(black_image_arr(mar_si_)).resize((width_arr_,1))
    arr_temp_ver_ = np.asarray(fig_temp_ver_)
    arr_temp_hor_ = np.asarray(fig_temp_hor_)
    for j_si_ in range(len(mar_si_[0])):
        stor_si_ = []
        coun_si_ = 0
        temp_house_ver_ = arr_temp_ver_
        temp_house_hor_ = arr_temp_hor_
        for i_si_ in range(len(mar_si_)-1):
            if color_test(mar_si_[i_si_][j_si_],r=1,b=0,w=1) != 0:
                new_si_[coun_si_][j_si_] = mar_si_[i_si_][j_si_]
                coun_si_ += 1
        coun_si_ = 0
    # Retorna uma matriz pequena e compacta
    return new_si_


# =================================================================
# ========== Inicio das interacoes com as funcoes =================
# =================================================================

# Essa parte abre o arquivo da imagem e aloca esta em fig
fig = im.open('meteor_challenge_01.png')

# Aplicando a funcao anterior que vai nos dar saidas para 
#comparar se esta tudo certo
#star1, meteor1, water1 = star_meteor(fig,1)

# Branco possui todas as cores pelo espectro de cor
# entao sera #FFFFFF
#print('Estrela = ', star1)

# Vermelho tem o maximo em vermelho e o resto zerado
# sendo entao #FF0000
#print('Meteoro = ', meteor1)

# Azul tem o maximo em azul e o resto zerado sendo
# entao #0000FF
#print('Water = ', water1)

# Aqui temos o tamanho da figura, muito util nessa task 
# para conferir se esta tudo certo
#w, h = fig.size
#print(w,h)

# Transformando a figura em um array para poder
# encontrar os meteoros "marinhos"
mar = np.asarray(fig)

# Caso precise transformar em imagem
#<var_da_imagem> = im.fromarray(np.uint8(<var_do_array>))


# Aplicando a funcao para reduzir a imagem e transformando 
# o array em imagem
red_arr = reducerr(mar)
red_fig = im.fromarray(np.uint8(red_arr))
   

# Aplicando a funcao que vai nos dar saidas para 
#comparar se esta tudo certo
#star2, meteor2, water2 = star_meteor(red_fig,1)

# Branco possui todas as cores pelo espectro de cor
# entao sera #FFFFFF
#print('Estrela = ', star2)

# Vermelho tem o maximo em vermelho e o resto zerado
# sendo entao #FF0000
#print('Meteoro = ', meteor2)

# Azul tem o maximo em azul e o resto zerado sendo
# entao #0000FF
#print('Water = ', water2)

# Aqui temos o tamanho da figura, muito util nessa task 
# para conferir se esta tudo certo
#w, h = red_fig.size
#print(w,h)

# Esse comparativo serviu para notificar se as informacoes 
# estiverem erradas
#if  star2 != star1 or meteor2 != meteor1:
#    print('Oh no') #fazer interrupção


# Aplicando a funcao para alterar as cores e transformando 
# o array em imagem
red_black_arr = white_red_black(red_arr)
red_black_fig = im.fromarray(np.uint8(red_black_arr))

# Image open/converter/save
abridor(red_black_arr,'imagem_p5.png')

# Aplicando a funcao anterior que vai nos dar saidas para 
#comparar se esta tudo certo
#star3, meteor3, water3 = star_meteor(red_black_fig,1)

# Branco possui todas as cores pelo espectro de cor
# entao sera #FFFFFF
#print('Estrela = ', star3)

# Vermelho tem o maximo em vermelho e o resto zerado
# sendo entao #FF0000
#print('Meteoro = ', meteor3)

# Azul tem o maximo em azul e o resto zerado sendo
# entao #0000FF
#print('Water = ', water3)

# Aqui temos o tamanho da figura, muito util nessa task 
# para conferir se esta tudo certo
#w, h = red_black_fig.size
#print(w,h)

# Esse comparativo serviu para notificar se as informacoes 
# estiverem erradas
#if  star2 != star1 or meteor2 != meteor1 or meteor3 != meteor1:
#    print('Oh no') #fazer interrupção

# Esta parte faz um novo para conferir se so sobrou 4 informacoes
# Preto, Branco, Vermelho e Azul
#li_li = dic(int)
#for pix_ in red_black_fig.getdata():
#     li_li[pix_] += 1
#print(li_li)


# Utilizando a função para deixar a figura pequena
all_tog_arr = short_image(red_black_arr)

# Removendo o solo entre agua e primeira estrela
last_white_or_red_ = 0
white_ = (255,255,255,255)
red_ = (255,0,0,255)
for i_ in range(len(all_tog_arr)):
    for j_ in range(len(all_tog_arr[0])):
        if np.all(white_ == all_tog_arr[i_][j_]) or np.all(red_ == all_tog_arr[i_][j_]):
            last_white_or_red_ = i_
            break

# Image open/converter/save
abridor(all_tog_arr,'imagem_p6.png')
            
coun = len(all_tog_arr) - last_white_or_red_ - 2
for i_ in reversed(range(1,coun)):
    deleter_ = last_white_or_red_ + i_
    all_tog_arr = np.delete(all_tog_arr,(deleter_), axis=0)


# Image open/converter/save
abridor(all_tog_arr,'aliens_aquaticos.png')
abridor(all_tog_arr,'imagem_p7.png')

water_aliens = 0
for j_ in range(len(all_tog_arr[0])):
    summ = 0
    for i_ in range(len(all_tog_arr)):
        summ += color_test(all_tog_arr[i_][j_],r=1,b=0,w=0)
    if(color_test(all_tog_arr[i_][j_],r=0,b=1,w=0) == 2):
        water_aliens += summ
#print('water_aliens', water_aliens)

# Saida de informação
ex_star, ex_meteor = star_meteor(im.fromarray(np.uint8(all_tog_arr)))
print('==========================================================')
print(f'=        Number of Stars           -        {ex_star}          =')
print(f'=        Number of Meteors         -        {ex_meteor}          =')
print(f'=  Meteors falling on the Water    -        {water_aliens}          =')
print('=        Hidden Phrase             -      not found      =')
print('==========================================================')


# Parte das letras daqui para frente

# Opções de melhoria e funções que podem ser uteis

# A partir desse ponto foi a tentativa de decodificação do texto

In [ ]:
## Parte que diz respeito a tentativa de achar palavras

# Criando um array para não alterar o anterior
# esse será modificado horizontalmente
f_array = all_tog_arr


# Tira as duas ultimas linhas do array, 
# deixando apenas as dos meteoros e estrelas
espaco = 0
while len(f_array) != 2:
    f_array = np.delete(f_array,(2), axis=0)


# Retira o espaço extra e coloca nome como novo
# mantem o antigo para o caso de os espaços entre
# os pontos ser importante!
novo = [[]]
novo.append([])
for i_ in range(len(f_array[0])-1):
    if i_ == len(f_array[0]):
        break
    if not(np.all(f_array[0][i_] == (0,0,0,255)) and np.all(f_array[0][i_+1] == (0,0,0,255))):
        novo[0].append(f_array[0][i_])
        novo[1].append(f_array[1][i_])


# Coloca a ultima linha preta para ser zero depois
# util para delimitar uma palavra em dash & dot
if np.any(novo[0][len(novo[0])-1] != [0,0,0,255]):
    novo[0].append([0,0,0,255])
    novo[1].append([0,0,0,255])


# Conta o numero de pretos, para saber quantos espaços
# existem, foram encontrados 181 - 6 com 7 informacoes
# temos os 177
cont_espacamento = 0
preto = [0,0,0,255]
for i in range(len(novo[0])):
    if np.all(novo[0][i] == preto):
        cont_espacamento += 1
print(cont_espacamento)


# Reduz de arrayy com quatro informacoes para
# apenas uma dependendo da cor se a cor for
#  preta [ 0, 0, 0, 255] em 0 e se for red ou branco
# [255,0,0,255] ou [255,255,255, 255] em 1
def testador(in_):
    # 0 é igual a preto
    # 1 é igual a branco
    out_ = 1
    if(np.all(in_ == [0,0,0,255])):
        out_ = 0
    return out_


# Esta parte cria um vetor para que a informacao
# anterior não seja perdida ou modificada
# mesmo que poderia alterar as informacoes
# da variavel, mas achei melhor ter um nome novo
# Faz 2 array com apenas zeros e uns
temp = [[]]
temp.append([])
for i_ in range(len(novo[0])-1):
    temp[0].append(testador(novo[0][i_]))
    temp[1].append(testador(novo[1][i_]))


# Transforma coluna em 0 ou 1 dependendo da situação
def dot_tra(a_, b_, dt_=0):
    # logica 0 dt_= 0, tudo que existe apenas na linha 0
    # vira dot (.) os que tem na 1 e 0 viram dash (-)
    # logica 1 dt_= 1, tudo que existe apenas na linha 0
    # vira dash (-) os que tem na 1 e 0 viram dot (.)
    #
    # Visualmente esta me pareceu mais correta, porem não
    # foi encontrado nenhuma frase que fizera sentido
    out_ = 0
    if a_ == 0:
        out_ = ' '
    else:
        if dt_ == 0:
            if b_ == 0:
                out_ = '.'
            else:
                out_ = '-'
        else:
            if b_ == 0:
                out_ = '-'
            else:
                out_ = '.'
    return out_


# Esta parte que realmente transforma em dot e traço
morse0_ = ''
morse1_ = ''
for i_ in range(len(temp[1])-2):
    # Usa a função criada para transformar, entra com
    # 0 no primeiro, para seguir a logica 0 e 1 na 
    # segunda para seguir a logica 2
    morse0_ += dot_tra(temp[0][i_+1],temp[1][i_+1],0)
    morse1_ += dot_tra(temp[0][i_+1],temp[1][i_+1],1)
print(morse0_)
print(' ')
print(morse1_)


In [ ]:
# Tira se tiver mais que 6 posicoes brancas seguidas
# em codigo morse não existe 7 seguidos, se tiver
# precisa ser tirado
# Esse foi o ponto em que parei o código para focar
# no relatório
#def tira_branco()
posicoes = []
for i_tb_ in range(len(temp[0])-8):
    ok_ = 0
    for j_tb_ in range(7):
        if temp[0][i_tb_ + j_tb_] == 0:
            ok_ = 1
    if ok_ == 0:
        for j_tb_ in range(20):
            posicoes.append([i_tb_ + j_tb_])
            if temp[0][i_tb_ + j_tb_] == 0:
                break



# Tentando por outro caminho fazer o dot-dash
def cansei(mar_c_,dd_=0):
    gan_temp = ''
    for i_ in range(len(mar_c_[0])-1):
        if np.all(mar_c_[0][i_] == [0,0,0,255]):
            gan_temp += ' '
        elif dd_ == 1:
            if np.all(mar_c_[1][i_] == [0,0,0,255]):#dot
                gan_temp += '.'
            else:
                gan_temp += '-'
        else:
            if np.all(mar_c_[1][i_] == [0,0,0,255]):#dash
                gan_temp += '-'
            else:
                gan_temp += '.'
    return gan_temp


lin1_ = all_tog_arr
while len(lin1_) != 2:
    lin1_ = np.delete(lin1_,(2), axis=0)


aaaaa = cansei(lin1_,0)
bbbbb = cansei(lin1_,1)


for i_ in range(len(lin1_[0])-1):
    if i_ == len(lin1_[0]):
        break
    if not(np.all(lin1_[0][i_] == [0,0,0,255]) and np.all(lin1_[0][i_+1] == (0,0,0,255))):
        novo[0].append(lin1_[0][i_])
        novo[1].append(lin1_[1][i_])

In [ ]:
# Python program to implement Morse Code Translator
  
'''
VARIABLE KEY
'cipher' -> 'stores the morse translated form of the english string'
'decipher' -> 'stores the english translated form of the morse string'
'citext' -> 'stores morse code of a single character'
'i' -> 'keeps count of the spaces between morse characters'
'message' -> 'stores the string to be encoded or decoded'
'''
  
# Dictionary representing the morse code chart
MORSE_CODE_DICT = { 'A':'.-',     'B':'-...',
                    'C':'-.-.',   'D':'-..',    'E':'.',
                    'F':'..-.',   'G':'--.',    'H':'....',
                    'I':'..',     'J':'.---',   'K':'-.-',
                    'L':'.-..',   'M':'--',     'N':'-.',
                    'O':'---',    'P':'.--.',   'Q':'--.-',
                    'R':'.-.',    'S':'...',    'T':'-',
                    'U':'..-',    'V':'...-',   'W':'.--',
                    'X':'-..-',   'Y':'-.--',   'Z':'--..',
                    '1':'.----',  '2':'..---',  '3':'...--',
                    '4':'....-',  '5':'.....',  '6':'-....',
                    '7':'--...',  '8':'---..',  '9':'----.',
                    '0':'-----',  ', ':'--..--', '.':'.-.-.-',
                    '?':'..--..', '/':'-..-.',   '-':'-....-',
                    '(':'-.--.',  ')':'-.--.-'}

# Function to encrypt the string
# according to the morse code chart
def encrypt(message):
    cipher = ''
    for letter in message:
        if letter != ' ':
  
            # Looks up the dictionary and adds the
            # correspponding morse code
            # along with a space to separate
            # morse codes for different characters
            cipher += MORSE_CODE_DICT[letter] + ' '
        else:
            # 1 space indicates different characters
            # and 2 indicates different words
            cipher += ' '
  
    return cipher
  
# Function to decrypt the string
# from morse to english
def decrypt(message):
    # extra space added at the end to access the
    # last morse code
    while message [0] == ' ':
        message = message[1:]
    cos2 = 0
    posis = 0
    for letter in message:
        if letter != ' ':
            posis += 1
            cos2 += 1
        if cos2 > 6:
            while message[posis] != ' ':
                posis += 1
                cos2 += 1
            for i in range(0,cos2):
                message = message.replace(message[i],' ')
#            tempori = posis - cos2
 #           message = message[:tempori,pos:]
    
    message += ' '
  
    decipher = ''
    citext = ''
    for letter in message:
        # checks for space
        if (letter != ' '):
  
            # counter to keep track of space
            i_cm = 0
  
            # storing morse code of a single character
            citext += letter
  
        # in case of space
        else:
            # if i = 1 that indicates a new character
            i_cm += 1
  
            # if i = 2 that indicates a new word
            if i_cm == 2 :
  
                 # adding space to separate words
                decipher += ' '
            else:
  
                # accessing the keys using their values (reverse of encryption)
                decipher += list(MORSE_CODE_DICT.keys())[list(MORSE_CODE_DICT
                .values()).index(citext)]
                citext = ''
  
    return decipher
  
#message = "GEEKS-FOR-GEEKS"
#result = encrypt(message.upper())
#print (result)

#message = "--. . . -.- ... -....- ..-. --- .-. -....- --. . . -.- ... "
#result = decrypt(message)
#print (result)
  
MORSE_PARA_TESTE = ".-.. ..- .. ...  . -.. ..- .-. .- -.. ---  -. . .--  . -- .--. .-.. --- -.-- . ."
#resultado_esperado = " "

In [ ]:
# Resultados que foram encontrados obs nada conclusivos
message0 = ".-. -|--. . . .-. .-. --. ... - ... .- ... -. -...|.-. ... .- . -- ....|--. ..- --... - --.- . -- ---- --- -.- - -- . .. -- - --...- .- . .-. . --.. - -- -.-- --. - - - -- . . -- -... --. - - ... .- . .. -. .. - --. - . .-. ... .-. .. .- .. . -- ...- --. .- -- -..-|--. -.. - -- ... -- ---- --- .-. .-. ... -- . . --. .. --. .. .- .. - .-.. . .- --.- .-. . . ... . -- ..-- -- .- -- -.-. .-.. . .- ... -. ... --. . . .- .. -- . . -- -..- --. . . - .... -.. .-|-- .- .-. .. -- ...- --. ..- --. - . --. . . .-. . .- .. .|--- -. -- . - -- --. -- .-.. .- .... -- .-- -- .-"
message1 = ".-. - --. . . .-. .-. --. ... - ... .- ... -. -... .-. ... .- . -- .... --. ..- --... - --.- . -- ---- --- -.- - -- . .. -- - --...- .- . .-. . --.. - -- -.-- --. - - - -- . . -- -... --. - - ... .- . .. -. .. - --. - . .-. ... .-. .. .- .. . -- ...- --. .- -- -..- --. -.. - -- ... -- ---- --- .-. .-. ... -- . . --. .. --. .. .- .. - .-.. . .- --.- .-. . . ... . -- ..-- -- .- -- -.-. .-.. . .- ... -. ... --. . . .- .. -- . . -- -..- --. . . - .... -.. .- -- .- .-. .. -- ...- --. ..- --. - . --. . . .-. . .- .. . --- -. -- . - -- --. -- .-.. .- .... -- .-- -- .-"
m1op1 = 'RT#GEERRGSTSASNB#RSAEMH#GU7TQEMĤOKTMEIMT¡AEREZTMYGTTTMEEMBGTTSAEINITGTERSRIAIEMVGAMX#GDTMSMĤORRSMEEGIGIAITLEAQREESEMÜMAMCLEASNSGEEAIMEEMXGEETHDA#MARIMVGUGTEGEEREAIE#ONMETMGMLAHMWMA'
m2op1 = 'KE#UTTKKUOEONOAJ#KONTIĤ#UG2EFTIHSREITMIE#NTKTÜEILUEEEITTIJUEEONTMAMEUETKOKMNMTIÓUNIP#UWEIOIHSKKOITTUMUMNMEYTNFKTTOTIZINIÄYTNOAOUTTNMITTIPUTTEĤWN#INKMIÓUGUETUTTKTNMT#SAITEIUIYNĤIDIN'
m1op2 = 'rtgeerrgstsasnbrsaemhgu7tqem?oktmeimt?aereztmygtttmeembgttsaeinitgtersriaiemvgamxgdtmsm?orrsmeegigiaitleaqreesem?mamcleasnsgeeaimeemxgeethdamarimvgugtegeereaieonmetmgmlahmwm?'
m2op2 = 'keuttkkuoeonoajkonti?ug2eftihsreitmie?ntkt?eilueeeittijueeontmameuetkokmnmti?unipuweioihskkoittumumnmeytnfkttotizini?ytnoaouttnmittiputte?wninkmi?uguetuttktnmtsaiteiuiyn?idi?'
message2 = "-.- .|..- - - -.- -.- ..- --- . --- -. --- .- .---|-.- --- -. - .. ----|..- --. ..--- . ..-. - .. .... ... .-. . .. - -- .. . ..---. -. - -.- - ..-- . .. .-.. ..- . . . .. - - .. .--- ..- . . --- -. - -- .- -- . ..- . - -.- --- -.- -- -. -- - .. ---. ..- -. .. .--.|..- .-- . .. --- .. .... ... -.- -.- --- .. - - ..- -- ..- -- -. -- . -.-- - -. ..-. -.- - - --- - .. --.. .. -. .. .-.- -.-- - -. --- .- --- ..- - - -. -- .. - - .. .--. ..- - - . ---- .-- -.|.. -. -.- -- .. ---. ..- --. ..- . - ..- - - -.- - -. -- -|... .- .. - . .. ..- .. -.-- -. ---- .. -.. .. -."
message3 = "-.- . ..- - - -.- -.- ..- --- . --- -. --- .- .--- -.- --- -. - .. ---- ..- --. ..--- . ..-. - .. .... ... .-. . .. - -- .. . ..---. -. - -.- - ..-- . .. .-.. ..- . . . .. - - .. .--- ..- . . --- -. - -- .- -- . ..- . - -.- --- -.- -- -. -- - .. ---. ..- -. .. .--. ..- .-- . .. --- .. .... ... -.- -.- --- .. - - ..- -- ..- -- -. -- . -.-- - -. ..-. -.- - - --- - .. --.. .. -. .. .-.- -.-- - -. --- .- --- ..- - - -. -- .. - - .. .--. ..- - - . ---- .-- -. .. -. -.- -- .. ---. ..- --. ..- . - ..- - - -.- - -. -- - ... .- .. - . .. ..- .. -.-- -. ---- .. -.. .. -."
